In [16]:
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image

In [10]:
#while True:
    # ret, frame3 = video.read()

    # frame4 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)
    # #frame = cv2.blur(frame4, (100,1))

    # frame = cv2.adaptiveThreshold(frame4, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 7)

    # cv2.imshow("vid", frame)

    # if cv2.waitKey(4) & 0xFF == ord('q'):
    #     break

# video.release()
# cv2.destroyAllWindows()

In [17]:
image2 = cv2.imread("image_samples/word_search_snacks.png")
image = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
#ret, image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY_INV)
image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 33, 25)

image = cv2.blur(image, (2,2))

contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

area = 0

for cnt in contours:
    area += cv2.contourArea(cnt)

area = area/len(contours)

letter_grid_coordinates = dict()
y_top, y_bottom = -1,-1

for i in range(len(contours)):
    #cv2.drawContours(image2, cnt, -1, (255,0,0), 7 )
    if cv2.contourArea(contours[i]) < 0.1*area:
        continue
    x, y, w, h = cv2.boundingRect(contours[i])
    #cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)

    pushed = False

    for key in letter_grid_coordinates.keys():
        if y + w//2 in range(key[0], key[1]):
            y_top, y_bottom = y, y+h
            letter_grid_coordinates[key].append((x - int((w)*(0.25)), y - int(h*(0.25)), x + w + int(w*(0.25)), y + h + int(h*(0.25))))
            pushed = True

    if not pushed:
        letter_grid_coordinates[(y, y+w)] = [(x - int(w*(0.25)), y - int(h*(0.25)), x + w + int(w*(0.25)), y + h + int(h*(0.25)))]
    
    #cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)
    

In [18]:
# video = cv2.VideoCapture(0)

# while True:
#     r, image2 = video.read()
#     image = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
#     #ret, image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY_INV)
#     image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 33, 25)

#     image = cv2.blur(image, (2,2))

#     contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     area = 0

#     for cnt in contours:
#         area += cv2.contourArea(cnt)

#     area = area/len(contours)

#     for i in range(len(contours)):
#         #cv2.drawContours(image2, cnt, -1, (255,0,0), 7 )
#         if cv2.contourArea(contours[i]) < 0.1*area:
#             continue
#         x, y, w, h = cv2.boundingRect(contours[i])
#         cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)

#     cv2.imshow("hsf", image2)
#     cv2.imshow("hsf2", image)

#     cv2.waitKey(80)

In [24]:
model = tf.keras.models.load_model("font_identifier.keras")

In [ ]:
cv2.imshow("hsf", image2)
cv2.imshow("hsf2", image)
cv2.waitKey(1000)
cv2.destroyAllWindows()

crossword = list()

for i in letter_grid_coordinates.keys():
    letter_grid_coordinates[i] = sorted(letter_grid_coordinates[i], key=lambda x: x[0])
    #print(letter_grid_coordinates[i])

for i in reversed(letter_grid_coordinates.values()):
    x = list()
    for j in i:
        x1, y1, x2, y2 = j
        img = image[y1:y2, x1:x2]
        img = Image.fromarray(img)
        img.thumbnail((28, 28), Image.Resampling.LANCZOS)

        pillow_image = Image.new("L", (28, 28), 0)
        pillow_image.paste(img, ((28 - img.size[0]) // 2, (28 - img.size[1]) // 2))

        img = np.array(pillow_image).reshape((1,28,28,1))
        img = img/255.0
        prediction = model.predict(img)
        x.append(chr(np.argmax(prediction) + 65))



    crossword.append(x)

# j = list(letter_grid_coordinates.values())[8][17]

# x1, y1, x2, y2 = j
# img = image[y1:y2, x1:x2]
# img = cv2.resize(img, (28, 28))
# cv2.imshow(str(x1), img)
# img = img.reshape((-1,28,28,1))
# img = img/255.0
# prediction = model.predict(img)
# print(chr(np.argmax(prediction) + 65))

# for i in letter_grid_coordinates.keys():
#     print(len(letter_grid_coordinates[i]) , "elements in", i)


error: OpenCV(4.12.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/core/src/matrix_c.cpp:190: error: (-5:Bad argument) Unknown array type in function 'cvarrToMat'


In [32]:
for i in crossword:
    print(' '.join(i))

Z W N R O C P O P G A I S W L U W X K L Y X Q Y
F N X S S L H Z R M J E W I S O T I R O D P C B
N J D P Z U B O X I L J M U Q C M E E G N A R O
T S E I K O O C Q G H G L C W Z S R E K C A R C
P I Q H Q L I T N V P O P A D O S M R C K W U C
S N Z C K H I I M G U M M Y B E A R S H O O X F
E N C O N R R S A H T J F R E N C H F R I E S
I I
D M R T Q P P L E P J W G E L L M L Z J C N I S
A K W A I I I G R P Y M S F I O Y T Y V K U G P
C V A T P U Q A C Q A A A C W N M I N J K O I T
K T S O E I X A E H T R O W P Y Y D N A C Z K Z
F R Y P A X K X C A X R G Q K E V C W S Z A N B
U Z N E N P I H I G I O C Y H T I R W A U L D S
J S F X U K R D Y C U F X E K K T E R P L M X B
V L B J T Q N E E D K K S J G P F O D V Y O E K
D E C Q S Y S E M X L K N B A O L R T O B N S J
G Z D K S N S S P H P K I S Y L D C J F E D C R
F T V W A T E R M E L O N E S Y E M U C E S B G
I E I A J K M N O U G X I L N L I U A R T A K M
R R Z Z P I Q X A W J A J K M H S O T E E H C Q
L P R P V P X O M C O G N C R Z B V Z 